### Tellier Thomas - Turp Henry - Turquetil Antoine

# SD207 - Data Challenge

## I - Un problème de classification

Il s'agissait de classifier des messages écrits pour des réseaux sociaux comme insultant envers une personne ou non. Le training set contenait 4415 échantillons et le test, 4414, et l'évaluation était simplement le pourcentage de bonne classification. Les librairies externes de machine learning n'étaient pas autorisées.

On a assez vite remarqué que les données sont assez réduites. Par exemples, 30 échantillons représentent 0.67% du test, ce qui est supérieur à la différence finale entre le premier et le second sur le test public. Il était donc difficile de valider son score avec les méthodes usuelles sans soumettre, la variance était très élevée. Par ailleurs, les données venaient d'un challenge Kaggle :

https://www.kaggle.com/c/detecting-insults-in-social-commentary

Les vainqueurs étaient d'accord après la compétition sur le fait que le hasard avait grandement décidé des premières places. C'est donc plus dans un esprit de découverte que de compétition acharnée que nous avons envisagé ce challenge.

## II - Stratégie et implémentation

Nous avons dès le début décidé d'implémenter notre challenge en C++, et de l'héberger sur github :

https://github.com/antoinux/data-challenge-winners

Nous avons utilisé deux librairies externes : Armadillo pour les opérations matricielles rapides et Boost pour les regex. Armadillo est une librairie regroupant la plupart des libraires d'algèbre linéaires classiques (OpenBlas, Lapack, Arpack, SuperLU...) sous une interface d'utilisation de style Matlab plutôt performante.

De manière générale, notre stratégie a été de tester les différentes approches sur scikit-learn et de sélectionner la meilleure à partir du score de validation croisée avant d'implémenter le strict nécessaire en C++. Pour l'implémentation, on se réferrait à la source et aux références de scikit pour être certain de ce qu'on faisait, et en comparant régulièrement les résultats des deux implémentations.

## III - Le parsing et le preprocessing

Nous avons commencé par retravailler un peu les données. Pour cela, nous avons procédé à grands coups d'expression régulières. Nous avons :
* enlevé tous les caractères spéciaux de type \x-- \u---- \n \r \t ... etc
* enlevé tous les liens URL
* enlevé les balises HTML
* enlevé la ponctuation, les chiffres et autres caractères spéciaux
* remplacé les smileys par un mot selon le type du smiley (positif/négatif)
* remplacé les suites de lettres identiques par la double lettre seulement (yaaaaaaay -> yaay)
* appliqué une liste de correctifs afin de limiter l'auto-censure (s..t -> shit par exemple)
* changé tous les mots de la famille 'you' en you (your youre -> you)

En ce qui concerne le stemming, on a essayé d'utiliser le stemmer de nltk, mais il ne s'est pas avéré plus efficace que de simplement tronquer les mots (du moins en validation). On s'en est donc finalement passé et l'implémentation est restée en C++ pur. Nous avons tout de même lemmatizé les mots en "you" avec une regex particulière.

Nous avons utilisé comme feature les mots seuls et les bi-grams.

## IV - Naive Bayes et la régression logistique

Le premier classifier implémenté fut Naive Bayes version "Bernoulli" :

http://scikit-learn.org/stable/modules/naive_bayes.html

Son avantage en terme d'implémentation est de se résumer à une série d'opérations matricielles de nécessitant pas d'optimiser une fonction de coût. Ca a quand même été plus difficile que prévu de faire aussi performant que scikit car on a été surpris de l'implémentation utilisée, inspirée par ce lien, et qui prend en compte les mots n'apparaissant pas dans les phrases :

http://nlp.stanford.edu/IR-book/html/htmledition/the-bernoulli-model-1.html

Aussi, utiliser les log-probas est mieux car permet d'éviter les instabilités numériques et donne d'agréables formules matricielles en transformant les produits de lois indépendantes en de belles sommes.

On s'est ensuite aperçu que la régression logistique alliée à une matrice de compte des occurances des mots/n-grams permettait était plus performante. En TP d'optimisation, on avait implémenté la méthode de Newton, ce qui n'a pas été d'une grande aide ici à cause du nombre de features des modèles (souvent supérieur à 10000). On s'est donc intéressé à l'implémentation de liblinear, sous-jacente à scikit-learn :

https://www.csie.ntu.edu.tw/~cjlin/papers/liblinear.pdf

Etant certains de la fonction de coût à optimiser, on a pu l'implémenter. La méthode d'optimisation a été une simple descente de gradient. Avec un pas constant, il arrivait que l'algorithme ne converge pas. On a remédier à ce problème en ajustant le pas à chaque étape suivant que la norme du gradient est en train de diminuer ou pas :

alpha *= (norm_grad > old_norm ? -1 : 1) * correct_rate + 1.;

Où alpha est le pas de la descente.

## V - Résumé de la méthode adoptée

Notre meilleure soumission a été obtenue en suivant ces étapes :

- Parsing
- Troncature à 5 caractères de tous les mots
- Stemming adapté aux mots de la famille "you"
- Matrice de compte des occurances sur les mots seuls et les Bi-grams
- Régression logistique
- Seuil de probabilité à 0.5

L'implémentation des n-grams sur les caractères augmente peut-être le score. L'ajustement du seuil après la régression logistique permet problement de gagner quelques points. Cet ajustement peut se faire simplement en maximisant le score sur la validation, ou de manière moins honnête en faisant beaucoup de soumissions avec des seuils différents.